# Gestructureerde data: preprocessing dataset

In dit voorbeeld gaan we verder werken op de titanic dataset bestudeerd in voorgaande notebook.
Hierbij gaan we preprocessing toevoegen of in meer detail bespreken.

Hierbij is een belangrijk onderscheid tussen pytorch en tensorflow op te merken. Hierbij is het belangrijk om te realiseren dat pytorch verwacht dat alle data dat uit de __get__item functie komt numeriek is. Deze moet niet noodzakelijk reeds geschaald zijn maar moet wel dat datatype hebben. Het is dus belangrijk om een goede keuze te maken op welke plaats de nodige preprocessing stappen uitgevoerd worden. 
In het geval van tensorflow (in het geval van mixed datatypes) plaatsen we van in het begin elke feature apart. Daarna voegen we gelijkaardige features samen om deze te preprocessen. Pas op het einde van dit proces worden alle features samengevoegd tot een numerieke tensor.

### Pytorch

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # om null-waarden in te vullen

# Load Titanic dataset
titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
display(titanic)

numeric_features = ['age', 'n_siblings_spouse', 'parch', 'fare']
numeric_transformer= Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categoric_features = ['sex', 'class', 'embark_town', 'deck', 'alone']
categoric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # median kan problemen geven met categorieke waarden
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categoric_transformer, categoric_features),

# dit bevat de preprocessing stappen dus heel sterk afhankelijk van de dataset waarmee je werkt
class TitanicDataset(Dataset):
    def __init__(self, dataframe, preprocessor):
        X = dataframe.drop('survived', axis=1)
        y = dataframe['survived']

        preprocessor.fit(X)
        X = preprocessor.transform(X)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y_values, dtype=torch.long)
        self.preprocessor = preprocessor
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # deze functie return de rij met index idx
        return self.X[idx], self.y[idx] # return inputs, labels

titanic_train = df.sample(frac=0.8)
titanic_test = titanic.drop(titanic_train.index)

dataset_train = TitanicDataset(titanic_train, preprocessor)
dataset_test = TitanicDataset(titanic_test, dataset_train.preprocessor)

## Oefening: Inkomensdataset

Met onderstaande code downloaden we een csv met gegevens rond het inkomen van volwassenen.
Plaats in de code-cellen eronder de nodige code om de volgende preprocessingstappen uit te voeren met pytorch:

* Splits in train-test
* Label kolom is het inkomen -> voer hiervoor ordinal encoding uit
* Voer normalisatie uit van de numerieke kolommen zodat alle waarden tussen 0 en 1 liggen (geen standaardverdeling)
* Voer ordinal encoding uit op de occupation en native_country kolom
* Voer one-hot encoding uit op de overige niet-numerieke kolommen

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 
           'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 
           'hours_per_week', 'native_country', 'income']
adult_income = pd.read_csv(url, header=None, names=columns)